In [43]:
from bertopic import BERTopic
import pandas as pd
import re
import spacy
import fitz
import os
import plotly.io as pio
import matplotlib.pyplot as plt
import seaborn as sns

In [25]:
df = pd.read_csv('../dataset/anfragen_oldenburg_detailed.csv')

# Pre-processing data

In [26]:
df['Beschreibung_original'] = df['Beschreibung']
# lowercasing
df['Beschreibung'] = df['Beschreibung'].str.lower()

# removal of punctuation and digits
def clean_entries(entry):
    entry = re.sub(r'[\d]', ' ', entry) # remove digits
    entry = re.sub(r'[^\w\s]', ' ', entry) # remove non-word characters
    return entry
df['Beschreibung'] = df['Beschreibung'].apply(clean_entries)

# removal of stopwords (e.g., der, die, das, ein)
nlp = spacy.load("de_core_news_sm")
def remove_stopwords(entry):
    doc = nlp(entry)
    return " ".join([token.text for token in doc if not token.is_stop and not token.is_punct and not token.is_space])
df['Beschreibung'] = df['Beschreibung'].apply(remove_stopwords)



In [27]:
# remove street names
street_doc = fitz.open('./Strassenverzeichnis-Oldb.pdf')
street_names = []
for i, page in enumerate(street_doc):
    lines = [line.strip() for line in page.get_text().splitlines() if line.strip()]
    
    if i == 0:
        lines = lines[11:]
    else:
        lines = lines[8:]

    # two special cases for long street names and one for weird formatting
    if i == 2:
        idx = lines.index('ANNETTE-VON-DROSTE-HUELSHOFF-')
        lines[idx] = lines[idx] + ' ' + lines[idx + 1]
        del lines[idx + 1]
    if i == 7:
        idx = lines.index('FRIEDRICH-CHRISTOPH-DAHLMANN-')
        lines[idx] = lines[idx] + ' ' + lines[idx + 1]
        del lines[idx + 1]
    if i == 15:
        idx = lines.index('LENZWEG')
        corrected = []
        corrected.append(lines[idx])
        corrected.extend(lines[idx + 4:idx + 6])
        corrected.extend(lines[idx + 10::3])
        street_names.extend(corrected)
        continue 
    streets = lines[::3]
    street_names += streets

# extend street names to account for ß and str. and take care of umlaute
street_names = [name.lower() for name in street_names]
# umlaute
def generate_umlaut_variants(name):
    variants = set()
    variants.add(name)

    replacements = [
        ('ae', 'ä'),
        ('oe', 'ö'),
        ('ue', 'ü'),
        ('ä', 'ae'),
        ('ö', 'oe'),
        ('ü', 'ue'),
    ]

    for a, b in replacements:
        if a in name:
            variants.add(name.replace(a, b))
        if b in name:
            variants.add(name.replace(b, a))
    
    return variants
street_names_ext = set()
name_patterns = []
for name in street_names:
    umlaut_names = generate_umlaut_variants(name)
    street_names_ext.update(umlaut_names)
for name in street_names_ext:
    ext_name = name
    if 'strasse' in name:
        ext_name = name.replace('strasse', r'\s?(strasse|straße|str\.?)')
    name_patterns.append(ext_name)

street_names_pattern = r'\b(?:' + '|'.join(name_patterns) + r')\b'
df['Beschreibung'] = df['Beschreibung'].str.replace(street_names_pattern, '', flags=re.IGNORECASE, regex=True)

In [28]:
# remove hausnummer, nr. etc.
hausnummer_pattern = r'\b(haus)?(nummer|nr\.?)\b'
df['Beschreibung'] = df['Beschreibung'].str.replace(hausnummer_pattern, '', flags=re.IGNORECASE, regex=True)

In [29]:
# remove stadtteile (stadtteile according to Wikipedia)
stadtteile = [
    'zentrum',
    'dobben',
    'haarenesch',
    'bahnhofsviertel',
    'gerichtsviertel',
    'ziegelhof',
    'ehnern',
    'bürgeresch',
    'buergeresch',
    'donnerschwee',
    'osternburg',
    'drielake',
    'everstehn',
    'hundsmühler höhe',
    'hundsmuehler hoehe',
    'thomasburg',
    'bloherfelde',
    'haarentor',
    'wechloy',
    'bürgerfelde',
    'buergerfelde',
    'rauhehorst',
    'dietrichsfeld',
    'alexandersfeld',
    'flugplatz',
    'ofenerdiek',
    'nadorst',
    'etzhorn',
    'ohmstede',
    'bornhorst',
    'neuenwege',
    'kloster blankenburg',
    'kreyenbrück',
    'kreyenbrueck',
    'bümmerstede',
    'buemmerstede',
    'tweelbäke west',
    'tweelbaeke west',
    'krusenbusch',
    'drielaker moor'
]
stadtteile_pattern = r'\b(?:' + '|'.join(re.escape(stadtteil) for stadtteil in stadtteile) + r')\b'
df['Beschreibung'] = df['Beschreibung'].str.replace(stadtteile_pattern, '', flags=re.IGNORECASE, regex=True)

In [30]:
# lemmatization (e.g., reducing gegangen to gehen)
def lemmatize(entry):
    doc = nlp(entry)
    return " ".join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct and not token.is_space])
df['Beschreibung'] = df['Beschreibung'].apply(lemmatize)

Removal of categories, e.g., Straßen or Wilde Müllkippe, would also be an option. In some sense, people are primed towards using these phrases, since they have previously selected categories. But removing e.g., Straßen would come at the expense of losing a lot of relevant information.

In [31]:
display(df['Beschreibung'].head())

0    Famila liegen rechts Müll Gebüsch bitte entfernen
1                 Ecke diverser Abfall illegal ablegen
2            Gang Höhe erneut illegal Abfall entsorgen
3    Containerstellplatz ecke diverser Hausmüll ent...
4                          Gefrier Kühlschrank ablegen
Name: Beschreibung, dtype: object

# Topic model

In [33]:
docs = df['Beschreibung'].dropna()

In [10]:
topic_model = BERTopic(language="german")
topics, probs = topic_model.fit_transform(docs)

In [34]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,240,-1_liegen_stehen_höhe_weg,"[liegen, stehen, höhe, weg, müll, gehweg, spie...",[befinden Absenkung mindestens cm breit cm Loc...
1,0,96,0_müll_wild_mülleimer_liegen,"[müll, wild, mülleimer, liegen, parkplatz, meh...",[erneut vermüllen Restmüll Sperrmüll blau Säck...
2,1,82,1_ampel_straße_signal_schlaglöch,"[ampel, straße, signal, schlaglöch, richtung, ...",[Ampel Defekt Grün leuchten wüsting kommend St...
3,2,71,2_fußball_werbung_aufkleber_unterhaltungsfußball,"[fußball, werbung, aufkleber, unterhaltungsfuß...",[Aufkleber Werbung Unterhaltungsfußball vfb Ol...
4,3,45,3_gefährlich_auto_straße_beschädigen,"[gefährlich, auto, straße, beschädigen, gefähr...",[schlaglöch Weg Straßenverkehr gefährden Zenti...
5,4,44,4_stehen_fahrzeug_kennzeichen_auto,"[stehen, fahrzeug, kennzeichen, auto, parkplat...",[mehrere Monat stehen Kurve Dersagauwegs alt P...
6,5,41,5_sperrmüll_gebotszeich_mast_drückampel,"[sperrmüll, gebotszeich, mast, drückampel, kap...",[Druckknopf Ampel Höhe hnr südlich Seite defek...
7,6,35,6_fahrrad_radfahrer_fahrradweg_rollstuhlfahrer,"[fahrrad, radfahrer, fahrradweg, rollstuhlfahr...",[Fußweg stehen stark regenfäll Bereichsweis Wa...
8,7,30,7_bushaltestellen_bus_schwelle_liegen,"[bushaltestellen, bus, schwelle, liegen, busha...",[Schliefenstraße Einmündung befinden quer Fahr...
9,8,29,8_glascontainer_müll_ecke_papier,"[glascontainer, müll, ecke, papier, altglas, g...",[Parkbucht Glascontainer Wilhelm Krüger strn s...


In [35]:
# Export the top 25 words for each topic
path = './topic_model_results'
file= os.path.join(path, 'top_words_per_topic.txt')
topic_info = topic_model.get_topic_info()
with open(file, 'w', encoding='utf-8') as f:
    for topic_id in topic_info[topic_info.Topic != -1].Topic:
        words = topic_model.get_topic(topic_id)
        f.write(f'Topic {topic_id}:\n')
        for word, score in words[:10]:
            f.write(f'{word} ({score:.4f})\n')
        f.write('\n')

In [39]:
topic_model.visualize_barchart(top_n_topics=20, topics=topic_info[topic_info.Topic != -1].Topic.tolist(), n_words=10)
#fig = topic_model.visualize_barchart(top_n_topics=20, topics=topic_info[topic_info.Topic != -1].Topic.tolist(), n_words=10)
#fig.write_image("./topic_model results/top_words_per_topic.png", width=1200, height=800)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': '#D55E00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.029917385608211687, 0.03209704372720085,
                    0.033931086447938935, 0.03507404424454232, 0.03680430491041335,
                    0.03879842703861958, 0.03924528475893372, 0.03980631915795096,
                    0.04043724712112707, 0.06999920378431643],
              'xaxis': 'x',
              'y': [entsorgen  , müllkippe  , abfall  , müllsäcke  , mehrere  ,
                    parkplatz  , liegen  , mülleimer  , wild  , müll  ],
              'yaxis': 'y'},
             {'marker': {'color': '#0072B2'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.02643675237723817, 0.02690526476694963,
                    0.028999957469920665, 0.030294381725741774,
                    0.03145006843003423, 0.037469301266646736,
                    0.041182919816943823, 0.049088711224438863,
                    0.06301190670088631, 0.0771930498370068],
              'xaxis': 'x2',
              'y': [sensor  , fußgänger  , stadteinwärts  , funktionieren  ,
                    leuchten  , richtung  , schlaglöch  , signal  , straße  , ampel
                    ],
              'yaxis': 'y2'},
             {'marker': {'color': '#CC79A7'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.06530375925620578, 0.08802046055011944, 0.09308569869822694,
                    0.15876107103988582, 0.17261410504114755, 0.17704440649923772,
                    0.17704440649923772, 0.17780274170225968, 0.18198144921681006,
                    0.18393306274876378],
              'xaxis': 'x3',
              'y': [verkehrszeich  , entfernen  , bitte  , oldenburg  , vfb  ,
                    gmbh  , unterhaltungsfußball  , aufkleber  , werbung  , fußball
                    ],
              'yaxis': 'y3'},
             {'marker': {'color': '#E69F00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.03135989141113486, 0.03135989141113486, 0.03355273771823561,
                    0.035388940486267347, 0.03647077748299165, 0.03912361712489884,
                    0.03919986426391858, 0.04380405184136061, 0.04803214441617477,
                    0.049743874736373324],
              'xaxis': 'x4',
              'y': [verkehrsschild  , ragen  , radfahr  , sicht  , führen  ,
                    gefährden  , beschädigen  , straße  , auto  , gefährlich  ],
              'yaxis': 'y4'},
             {'marker': {'color': '#56B4E9'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.043122679196653003, 0.047902557831779324,
                    0.04802655395027013, 0.04906224119439943, 0.04972997090424176,
                    0.06438175879124569, 0.06438175879124569, 0.07359336179159914,
                    0.07669858409006168, 0.08896131809887309],
              'xaxis': 'x5',
              'y': [parken  , anhänger  , öffentlich  , kennzeich  , abgemeldet  ,
                    parkplatz  , auto  , kennzeichen  , fahrzeug  , stehen  ],
              'yaxis': 'y5'},
             {'marker': {'color': '#009E73'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.04216610360713883, 0.04216610360713883, 0.04216610360713883,
                    0.051586457869798555, 0.05426677595991896, 0.06165105612809547,
                    0.07453741572511631, 0.07453741572511631, 0.07453741572511631,
                    0.08969356072194447],
              'xaxis': 'x6',
              'y': [beeteinfassunge  , baumhaus  , beklebter  , schild  , vfb  ,
                    kaputt  , drückampel  , mast  , gebotszeich  , sperrmüll  ],
              'yaxis': 'y6'},
             {'marker': {'color': '#F0E442'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.03178185762392762, 0.03178185762392762, 0.03178185762392762,
           

In [49]:
#topic_model.visualize_topics()
# Generate the interactive Plotly figure
fig = topic_model.visualize_topics()

# Export to HTML file (specify your desired path)
fig.write_html("./topic_model_results/topic_model_visualization.html")

In [56]:
topic_model.visualize_heatmap()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'coloraxis': 'coloraxis',
              'hovertemplate': 'x: %{x}<br>y: %{y}<br>Similarity Score: %{z}<extra></extra>',
              'name': '0',
              'type': 'heatmap',
              'x': [0_müll_wild_mülleimer, 1_ampel_straße_signal,
                    2_fußball_werbung_aufkleber, 3_gefährlich_auto_straße,
                    4_stehen_fahrzeug_kennzeichen, 5_sperrmüll_gebotszeich_mast,
                    6_fahrrad_radfahrer_fahrrad..., 7_bushaltestellen_bus_schwe...,
                    8_glascontainer_müll_ecke, 9_regenwasser_laufen_wasser,
                    10_brücke_liegen_autobahnbr..., 11_herr_grundschule_gemeinsam,
                    12_illegal_müllentsorgung_a..., 13_asphalt_schließen_cm,
                    14_sofa_ca_woche, 15_einmündung_unrat_äste,
                    16_grün_verkehr_pfeil, 17_baum_bäume_ewe,
                    18_containerstellplatz_ents..., 19_autobahn_müll_autobahnbr...,
                    20_foto_schild_sehen],
              'xaxis': 'x',
              'y': [0_müll_wild_mülleimer, 1_ampel_straße_signal,
                    2_fußball_werbung_aufkleber, 3_gefährlich_auto_straße,
                    4_stehen_fahrzeug_kennzeichen, 5_sperrmüll_gebotszeich_mast,
                    6_fahrrad_radfahrer_fahrrad..., 7_bushaltestellen_bus_schwe...,
                    8_glascontainer_müll_ecke, 9_regenwasser_laufen_wasser,
                    10_brücke_liegen_autobahnbr..., 11_herr_grundschule_gemeinsam,
                    12_illegal_müllentsorgung_a..., 13_asphalt_schließen_cm,
                    14_sofa_ca_woche, 15_einmündung_unrat_äste,
                    16_grün_verkehr_pfeil, 17_baum_bäume_ewe,
                    18_containerstellplatz_ents..., 19_autobahn_müll_autobahnbr...,
                    20_foto_schild_sehen],
              'yaxis': 'y',
              'z': {'bdata': ('+v9/P8Y6+D7cDY4+pA0SPzCCDD8PSQ' ... '8rP4JMGT8yWf8+WuLiPh5t/D4AAIA/'),
                    'dtype': 'f4',
                    'shape': '21, 21'}}],
    'layout': {'coloraxis': {'colorbar': {'title': {'text': 'Similarity Score'}},
                             'colorscale': [[0.0, 'rgb(247,252,240)'], [0.125,
                                            'rgb(224,243,219)'], [0.25,
                                            'rgb(204,235,197)'], [0.375,
                                            'rgb(168,221,181)'], [0.5,
                                            'rgb(123,204,196)'], [0.625,
                                            'rgb(78,179,211)'], [0.75,
                                            'rgb(43,140,190)'], [0.875,
                                            'rgb(8,104,172)'], [1.0,
                                            'rgb(8,64,129)']]},
               'height': 800,
               'hoverlabel': {'bgcolor': 'white', 'font': {'family': 'Rockwell', 'size': 16}},
               'legend': {'title': {'text': 'Trend'}},
               'margin': {'t': 60},
               'showlegend': True,
               'template': '...',
               'title': {'font': {'color': 'Black', 'size': 22},
                         'text': '<b>Similarity Matrix</b>',
                         'x': 0.55,
                         'xanchor': 'center',
                         'y': 0.95,
                         'yanchor': 'top'},
               'width': 800,
               'xaxis': {'anchor': 'y', 'constrain': 'domain', 'domain': [0.0, 1.0], 'scaleanchor': 'y'},
               'yaxis': {'anchor': 'x', 'autorange': 'reversed', 'constrain': 'domain', 'domain': [0.0, 1.0]}}
})

In [55]:
topic_model.visualize_hierarchy()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hoverinfo': 'text',
              'marker': {'color': 'rgb(61,153,112)'},
              'mode': 'lines',
              'type': 'scatter',
              'x': {'bdata': 'AAAAAAAAAACQpqx7bEXnP5CmrHtsRec/AAAAAAAAAAA=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAAAALsAAAAAAAAAuwAAAAAAAADnAAAAAAAAAOcA=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'marker': {'color': 'rgb(61,153,112)'},
              'mode': 'lines',
              'type': 'scatter',
              'x': {'bdata': 'kKase2xF5z8dxnnjYz/rPx3GeeNjP+s/AAAAAAAAAAA=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAAAANMAAAAAAAAA0wAAAAAAAgEHAAAAAAACAQcA=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'marker': {'color': 'rgb(61,153,112)'},
              'mode': 'lines',
              'type': 'scatter',
              'x': {'bdata': 'AAAAAAAAAAAyZvC4PkXtPzJm8Lg+Re0/HcZ542M/6z8=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAAAAFMAAAAAAAAAUwAAAAAAAgDvAAAAAAACAO8A=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'marker': {'color': 'rgb(61,153,112)'},
              'mode': 'lines',
              'type': 'scatter',
              'x': {'bdata': 'AAAAAAAAAADZkUOUR9nrP9mRQ5RH2es/AAAAAAAAAAA=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAACARsAAAAAAAIBGwAAAAAAAgEvAAAAAAACAS8A=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'marker': {'color': 'rgb(61,153,112)'},
              'mode': 'lines',
              'type': 'scatter',
              'x': {'bdata': 'AAAAAAAAAACN5OFY7FrsP43k4VjsWuw/AAAAAAAAAAA=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAABAUMAAAAAAAEBQwAAAAAAAwFLAAAAAAADAUsA=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'marker': {'color': 'rgb(61,153,112)'},
              'mode': 'lines',
              'type': 'scatter',
              'x': {'bdata': '2ZFDlEfZ6z/yg0dxGr3tP/KDR3Eave0/jeThWOxa7D8=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAAAAScAAAAAAAABJwAAAAAAAgFHAAAAAAACAUcA=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'marker': {'color': 'rgb(61,153,112)'},
              'mode': 'lines',
              'type': 'scatter',
              'x': {'bdata': 'MmbwuD5F7T+RfnlQsa/vP5F+eVCxr+8/8oNHcRq97T8=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAABAMMAAAAAAAEAwwAAAAAAAAE7AAAAAAAAATsA=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'marker': {'color': 'rgb(255,65,54)'},
              'mode': 'lines',
              'type': 'scatter',
              'x': {'bdata': 'AAAAAAAAAAC7l6DgUGXtP7uXoOBQZe0/AAAAAAAAAAA=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAABAVcAAAAAAAEBVwAAAAAAAwFfAAAAAAADAV8A=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'marker': {'color': 'rgb(35,205,205)'},
              'mode': 'lines',
              'type': 'scatter',
              'x': {'bdata': 'AAAAAAAAAABtmFVeP83oP22YVV4/zeg/AAAAAAAAAAA=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAABAWsAAAAAAAEBawAAAAAAAwFzAAAAAAADAXMA=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'marker': {'color': 'rgb(0,116,217)'},
              'mode': 'lines',
              'type': 'scatter',
              'x': {'bdata': 'u5eg4FBl7T978dm2whPwP3vx2bbCE/A/bZhVXj/N6D8=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAACAVsAAAAAAAIBWwAAAAAAAgFvAAAAAAACAW8A=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'text'

In [41]:
# export top documents per topic
file= os.path.join(path, 'top_docs_per_topic.txt')
representative_docs = topic_model.get_representative_docs()
with open(file, 'w', encoding='utf-8') as f:
    for topic_id, docs in representative_docs.items():
        if topic_id == -1:
            continue
        f.write(f'Topic {topic_id}:\n')
        for i, doc in enumerate(docs[:3]):
            f.write(f'Document {i+1}:\n')
            f.write(f'{doc.strip()}\n\n')
        f.write('\n\n')

In [42]:
# export original/unprocessed documents per topic
df['Topic'] = topics
file= os.path.join(path, 'top_original_docs_per_topic.txt')
with open(file, 'w', encoding='utf-8') as f:
    topic_info = topic_model.get_topic_info()
    for topic_id in topic_info[topic_info.Topic != -1].Topic:
        f.write(f"Topic {topic_id}:\n")

        top_docs = df[df["Topic"] == topic_id].head(3)

        for i, row in top_docs.iterrows():
            f.write(f"  Document {i}:\n")
            f.write(f"  {row['Beschreibung_original'].strip()}\n\n")

        f.write("\n\n")